## Reccomendation System: Netflix Movies & TV Series Recommendations 

## Introduction

In this notebook, I aim to create a recommendation system for TV series and Movies from this dataset. Machine Learning plays a crucial role in creating recommendation systems. Recommendation systems are used by many online platforms such as e-commerce websites, streaming services, social media platforms, and more to suggest items, products, movies, music, or content to users based on their preferences, behaviors, and patterns. In this notebook I am going to attempt to use collaborative filtering and user-entertainment interaction to find patterns and make predictions.

## Dataset Overview & Source

For this analysis, I will be using a dataset that I obtained from Kaggle, which is known for having many datasets that are used for research and analysis purposes. The dataset I got comprises of the following information: Type of Content, Country of Origin, Date Added, Release Year, Rating, and Duration. The dataset is useful for studying common trends and patterns within Netflix.

The dataset was sourced from Kaggle and it can be accessed using the following think: https://www.kaggle.com/datasets/lovishbansal123/netflix-dataset
The dataset was downloaded on 3/12/2024!

## Dataset Information 

Every row represents a single entry entertainment content available on Netflix. Whether it be a movie or a TV show. The dataset was created using public information from Netflix, using a web scrapper. Furthermore, as Netflix is constantly adapating and evolving, it's really unlikely that this dataset would include EVERY observations, so there are some limitations in this specific area. Because of this, the dataset could affect the reliability and generalizability of the findings. However, this dataset is still useful in analyzing the available data for stakeholders and strategies to keep viewers engaged creating the best possible outcomes for Netflix and it's shareholders!


# Imports 

In [11]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [12]:
df = pd.read_csv("netflix_titles.csv" , na_values="NaN")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


## Dataset Cleaning

In [13]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In this case dropping rows that might not have directors, cast,date added,rating and duration not listed might not be too much of an issue. For our model here our TF-IDF Vectorizer and Cosine Similarity are focusing on title, country, and description. So it would be approrpriate to drop the observations with one of those missing values but for the rest of the variables it should not hinder the accuracy of our recommendation system too much.

In [14]:
cleaned_df = df.dropna(subset=['country'])
cleaned_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
...,...,...,...,...,...,...,...,...,...,...,...,...
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [15]:
## Number of rows dropped
df.shape[0] - cleaned_df.shape[0]

831

## Splitting into two dataframes: Movies & TV Series

In [16]:
movies = cleaned_df[cleaned_df.type == 'Movie']
movies

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...
...,...,...,...,...,...,...,...,...,...,...,...,...
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [17]:
movies['combined'] = movies['title'] + ' ' + movies['description']+ ' ' + movies['country']
movies

/var/folders/51/d53d0_591f5fdkvr9fr8gsz00000gn/T/ipykernel_84154/3142620380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['combined'] = movies['title'] + ' ' + movies['description']+ ' ' + movies['country']


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,combined
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Dick Johnson Is Dead As her father nears the e...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...","Sankofa On a photo shoot in Ghana, an American..."
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,The Starling A woman adjusting to life after a...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...,Je Suis Karl After most of her family is murde...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...,Jeans When the father of the man she loves ins...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...,Zinzana Recovering alcoholic Talal wakes up in...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...","Zodiac A political cartoonist, a crime reporte..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Zombieland Looking to survive in a world taken...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...","Zoom Dragged from civilian life, a former supe..."


In [18]:
shows = cleaned_df[cleaned_df.type == 'TV Show']
shows

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
15,s16,TV Show,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,"September 22, 2021",2021,TV-MA,4 Seasons,"TV Comedies, TV Dramas",Students of color navigate the daily slights a...
17,s18,TV Show,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,"September 22, 2021",2020,TV-MA,2 Seasons,"Crime TV Shows, Spanish-Language TV Shows, TV ...",Strangers Diego and Isabel flee their home in ...
...,...,...,...,...,...,...,...,...,...,...,...,...
8780,s8781,TV Show,Yo-Kai Watch,NaN,"Johnny Yong Bosch, J.W. Terry, Alicyn Packard,...",United States,"April 1, 2016",2015,TV-Y7,1 Season,"Anime Series, Kids' TV",Nate frees a mythical being trapped in a magic...
8795,s8796,TV Show,Yu-Gi-Oh! Arc-V,NaN,"Mike Liscio, Emily Bauer, Billy Bob Thompson, ...","Japan, Canada","May 1, 2018",2015,TV-Y7,2 Seasons,"Anime Series, Kids' TV",Now that he's discovered the Pendulum Summonin...
8796,s8797,TV Show,Yunus Emre,NaN,"Gökhan Atalay, Payidar Tüfekçioglu, Baran Akbu...",Turkey,"January 17, 2017",2016,TV-PG,2 Seasons,"International TV Shows, TV Dramas","During the Mongol invasions, Yunus Emre leaves..."
8797,s8798,TV Show,Zak Storm,NaN,"Michael Johnston, Jessica Gee-George, Christin...","United States, France, South Korea, Indonesia","September 13, 2018",2016,TV-Y7,3 Seasons,Kids' TV,Teen surfer Zak Storm is mysteriously transpor...


In [19]:
shows['combined'] = shows['title'] + ' ' + shows['description']+ ' ' + shows['country']
shows


/var/folders/51/d53d0_591f5fdkvr9fr8gsz00000gn/T/ipykernel_84154/1142128145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shows['combined'] = shows['title'] + ' ' + shows['description']+ ' ' + shows['country']


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,combined
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...","Blood & Water After crossing paths at a party,..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Kota Factory In a city of coaching centers kno...
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,The Great British Baking Show A talented batch...
15,s16,TV Show,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,"September 22, 2021",2021,TV-MA,4 Seasons,"TV Comedies, TV Dramas",Students of color navigate the daily slights a...,Dear White People Students of color navigate t...
17,s18,TV Show,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,"September 22, 2021",2020,TV-MA,2 Seasons,"Crime TV Shows, Spanish-Language TV Shows, TV ...",Strangers Diego and Isabel flee their home in ...,Falsa identidad Strangers Diego and Isabel fle...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8780,s8781,TV Show,Yo-Kai Watch,NaN,"Johnny Yong Bosch, J.W. Terry, Alicyn Packard,...",United States,"April 1, 2016",2015,TV-Y7,1 Season,"Anime Series, Kids' TV",Nate frees a mythical being trapped in a magic...,Yo-Kai Watch Nate frees a mythical being trapp...
8795,s8796,TV Show,Yu-Gi-Oh! Arc-V,NaN,"Mike Liscio, Emily Bauer, Billy Bob Thompson, ...","Japan, Canada","May 1, 2018",2015,TV-Y7,2 Seasons,"Anime Series, Kids' TV",Now that he's discovered the Pendulum Summonin...,Yu-Gi-Oh! Arc-V Now that he's discovered the P...
8796,s8797,TV Show,Yunus Emre,NaN,"Gökhan Atalay, Payidar Tüfekçioglu, Baran Akbu...",Turkey,"January 17, 2017",2016,TV-PG,2 Seasons,"International TV Shows, TV Dramas","During the Mongol invasions, Yunus Emre leaves...","Yunus Emre During the Mongol invasions, Yunus ..."
8797,s8798,TV Show,Zak Storm,NaN,"Michael Johnston, Jessica Gee-George, Christin...","United States, France, South Korea, Indonesia","September 13, 2018",2016,TV-Y7,3 Seasons,Kids' TV,Teen surfer Zak Storm is mysteriously transpor...,Zak Storm Teen surfer Zak Storm is mysteriousl...


In [20]:
shows_us = shows[shows["country"]=="United States"]
shows_us

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,combined
15,s16,TV Show,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,"September 22, 2021",2021,TV-MA,4 Seasons,"TV Comedies, TV Dramas",Students of color navigate the daily slights a...,Dear White People Students of color navigate t...
40,s41,TV Show,He-Man and the Masters of the Universe,NaN,"Yuri Lowenthal, Kimberly Brooks, Antony Del Ri...",United States,"September 16, 2021",2021,TV-Y7,1 Season,"Kids' TV, TV Sci-Fi & Fantasy",Mighty teen Adam and his heroic squad of misfi...,He-Man and the Masters of the Universe Mighty ...
55,s56,TV Show,Nailed It,NaN,"Nicole Byer, Jacques Torres",United States,"September 15, 2021",2021,TV-PG,6 Seasons,Reality TV,Home bakers with a terrible track record take ...,Nailed It Home bakers with a terrible track re...
67,s68,TV Show,Saved by the Bell,NaN,"Mark-Paul Gosselaar, Tiffani Thiessen, Mario L...",United States,"September 15, 2021",1994,TV-PG,9 Seasons,"Kids' TV, TV Comedies","From middle school to college, best friends Za...",Saved by the Bell From middle school to colleg...
82,s83,TV Show,Lucifer,NaN,"Tom Ellis, Lauren German, Kevin Alejandro, D.B...",United States,"September 10, 2021",2021,TV-14,6 Seasons,"Crime TV Shows, TV Comedies, TV Dramas","Bored with being the Lord of Hell, the devil r...","Lucifer Bored with being the Lord of Hell, the..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8710,s8711,TV Show,Weeds,NaN,"Mary-Louise Parker, Hunter Parrish, Alexander ...",United States,"April 1, 2014",2012,TV-MA,8 Seasons,"TV Comedies, TV Dramas",A suburban mother starts selling marijuana to ...,Weeds A suburban mother starts selling marijua...
8726,s8727,TV Show,When Calls the Heart,NaN,"Erin Krakow, Daniel Lissing, Lori Loughlin, Ma...",United States,"December 29, 2018",2017,TV-PG,5 Seasons,TV Dramas,"Leaving behind her upper-crust city roots, a w...",When Calls the Heart Leaving behind her upper-...
8747,s8748,TV Show,Winsanity,NaN,Donald Faison,United States,"December 15, 2018",2016,TV-G,1 Season,Reality TV,Actor Donald Faison hosts this game show in wh...,Winsanity Actor Donald Faison hosts this game ...
8755,s8756,TV Show,Women Behind Bars,NaN,NaN,United States,"November 1, 2016",2010,TV-14,3 Seasons,"Crime TV Shows, Docuseries",This reality series recounts true stories of w...,Women Behind Bars This reality series recounts...


## Building a Recommendation System with Sci Kit Learns TfIdVectorizer and Cosine Similarity 

In [21]:
class MovieSearcher:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(data["combined"].tolist())

    def search(self, title):
        query_vec = self.vectorizer.transform([title])
        similarity = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        indices = np.argsort(similarity)[-6:-1]  # Get top 5 indices, excluding the highest (itself)
        results = self.data.iloc[indices].copy()
        results['similarity'] = similarity[indices]
        results = results.sort_values(by="similarity", ascending=False).drop(columns=["similarity"])
        return results

# Usage
movie_searcher = MovieSearcher(movies)
result = movie_searcher.search("Example Movie Title")
print(result)


     show_id   type            title        director  \
7930   s7931  Movie       San Pietro     John Huston   
5229   s5230  Movie      Bomb Scared   Borja Cobeaga   
6278   s6279  Movie         Being AP   Anthony Wonke   
4557   s4558  Movie  A Witches' Ball  Justin G. Dyck   
8224   s8225  Movie      The Brawler     Ken Kushner   

                                                   cast         country  \
7930                                                NaN   United States   
5229  Javier Cámara, Julián López, Miren Ibarguren, ...           Spain   
6278                                         Tony McCoy  United Kingdom   
4557  Morgan Neundorf, Karen Slater, Loukia Ioannou,...          Canada   
8224  Zach McGowan, Amy Smart, Joe Pantoliano, Burt ...   United States   

            date_added  release_year rating duration  \
7930    March 31, 2017          1945  TV-14   32 min   
5229  October 12, 2017          2017  TV-MA   89 min   
6278   August 15, 2016          2015  TV-14 

## Creating a Search Widget for Movies 

In [22]:
class MovieSearcher:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(data["combined"].tolist())

    def search(self, title):
        query_vec = self.vectorizer.transform([title])
        similarity = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        indices = np.argsort(similarity)[-6:-1]  # Get top 5 indices, excluding the highest (itself)
        results = self.data.iloc[indices].copy()
        results['similarity'] = similarity[indices]
        results = results.sort_values(by="similarity", ascending=False).drop(columns=["similarity"])
        return results

# Create an instance of MovieSearcher
movie_searcher = MovieSearcher(movies)

# Setup widgets
movie_input = widgets.Text(value='Grown Ups', description='Movie Title:', disabled=False)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = movie_searcher.search(title)
            display(results)

movie_input.observe(on_type, names='value')
display(movie_input, movie_list)

Text(value='Grown Ups', description='Movie Title:')

Output()

## Creating a Search Widget for TV Series 

In [23]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display


class TVShowSearcher:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(data["combined"].tolist())

    def search(self, title_or_description):
        query_vec = self.vectorizer.transform([title_or_description])
        similarity = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        indices = np.argsort(similarity)[-5:]  # Get top 5 indices
        results = self.data.iloc[indices]
        return results

# Usage
tv_show_searcher = TVShowSearcher(shows)

# Setup input widget
shows_input = widgets.Text(value="All American", description="TV Shows:", disabled=False)
show_list = widgets.Output()

def on_type_show(data):
    with show_list:
        show_list.clear_output()
        if len(data["new"]) > 5:
            results = tv_show_searcher.search(data["new"])
            display(results)

shows_input.observe(on_type_show, names='value')
display(shows_input, show_list)

    


Text(value='All American', description='TV Shows:')

Output()

# Conclusion 

This analysis has provided valuable insights into the trends and patterns of Netflix content based on genre and descriptive metadata. By leveraging techniques such as TF-IDF vectorization and cosine similarity, we were able to discern distinct preferences and trends that shape viewer recommendations. The findings suggest that certain genres and descriptors are more prevalently recommended, influenced by their representation and viewer popularity within the dataset.

Our study underscores the significance of sophisticated metadata handling and analysis in enhancing content recommendation systems. For stakeholders, including content curators and platform engineers, these insights can guide more nuanced content strategies and algorithm refinements. This could lead to a more engaging user experience, where recommendations are both more accurate and reflective of diverse viewer preferences.

Moving forward, further research could expand on this foundational work by incorporating additional metadata layers and exploring temporal changes in viewer behavior and content popularity. Such efforts would continue to refine our understanding of digital content consumption and the dynamic landscape of streaming platforms.